In [ ]:
# FIRST import all the necessary libraries and modules!
import cv2               # import OpenCV
import numpy as np       # import NumPy

# import functions 
import sys
sys.path.insert(0, '../..')
from utils import *    
def crop(img, top, bottom):
    imgCropped = np.zeros(img.shape, dtype=np.uint8)
    imgCropped[top:bottom] += img[top:bottom]
    return imgCropped


In [ ]:
screen_center = 320
def findGreatestContour(contours):
    cnt = [-1, -1]
    for i in range(0, len(contours)):
        if (cv2.contourArea(contours[i]) >= cnt[1]):
            cnt  = [i, cv2.contourArea(contours[i])]
    return contours[cnt[0]]

## need to define?
def turn(angle): 
    print(angle)

def getAngle2(x, y):
    dist = cx - screen_center
    
    if dist is not 0:
        ratio = dist/screen_center
        angle = ratio*c 
    return angle 
    
def getAngle(contour):
    
    M = cv2.moments(contour)
    cx = int(M['m10']/M['m00'])
    
    c = 15
    angle, ratio = (0, 0)

    dist = cx - screen_center

    if dist is not 0:
        ratio = dist/screen_center
        angle = ratio*c 
    return angle 

def get_angle(x, error=0.5):
    """
    input:
        1. box dimensions (list) --> x, y, w, h
    output:
        if 1 ft/more away:
            1. an angle for the car to turn
        else:
            1. None
    """ 
    ratio, angle, screen_center, c = (0, 0, 720.0, 2)
    
    dist = x - screen_center
    if box_dist is not 0:
        ratio = box_dist/screen_center 
    if abs(ratio) > error:
        angle = ratio*constant
    return angle

def lineFollowing(img, colorRange, testing=False):
    res =[None, None]
    if testing:
        res = [img, None]
    # print(img)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    mask = cv2.inRange(hsv, colorRange[0], colorRange[1])
    # inv_mask = cv2.bitwise_not(mask)
    ctImg, contours, __ = cv2.findContours(mask,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    if contours == []:
        res[-1] = "empty contour list"
        return res
    cnt = findGreatestContour(contours)
    if cv2.contourArea(cnt) > 5:
        res[-1] = "contour area too small"
        return res 
    x,y,w,h = cv2.boundingRect(cnt)
    center = (x+w/2, y+h/2)
    if testing:
        output = cv2.rectangle(img, (x,y),(x+w,y+h),(0,0, 255),2) 
        output = cv2.circle(output, center, 10, (0,0, 255), -1)
        res[0] = output

    angle = getAngle(cnt)
    turn(angle)
    return res 

def pathSelection(image, colorList):
    image = crop(image, 100, 200)
    for color in colorList:
        output, angle = lineFollowing(image, color) 
        if angle is not None:
            print(color)
            return [output, angle]
    return [image, -1]

# green, yellow 
c = 15
colorRange = [(np.array([70, 45, 30]), np.array([125, 140, 215])), 
             (np.array([0, 80, 95]), np.array([25, 200, 250]))]

# showVideoAction(pathSelection, colorRange)

############################################################################################################
# Sign detection functions

def getKeypoints(image, feature_detection_algorithm, hessianThreshold = 400):
    '''
    outputs keypoints and descriptions for the inputted image.
    '''
    global sift, surf, orb
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Create SIFT, SURF, or ORB objects
    if feature_detection_algorithm=='sift':
        sift = cv2.xfeatures2d.SIFT_create()
        kp, des = sift.detectAndCompute(image, None)

    elif feature_detection_algorithm=='surf':
        surf = cv2.xfeatures2d.SURF_create(hessianThreshold, extended=True)
        kp, des = surf.detectAndCompute(image, None)

    elif feature_detection_algorithm=='orb':
        orb = cv2.ORB_create(nFeatures)
        kp, des = orb.detectAndCompute(image, None)

    return kp, des

def signDetection(frame, queryKeypoints, queryDescriptions, feature_detection_algorithm, 
                  hessianThreshold = 400, MIN_MATCH_COUNT = 20, MIN_MATCH_PERCENTAGE = 0.18, 
                  MIN_MATCH = True):

    '''
    Input a frame and function will output a boolean. True = sign detected and 
    False = not detected.
    
    (Np.ndarray) frame
    (STRING) feature_detection_algorithm = 'sift', 'surf', 'orb'.
    (INT) hessianThreshold is basically number of key points. Smaller = more keypoints.
    (INT) MIN_MATCH_COUNT is the minimum number of good matches for an object to be 
        considered the query image.
    (FLOAT) MIN_MATCH_PERCENTAGE is the minimum match percentage for an object to be 
        considered the query image.
    (BOOL) MIN_MATCH is a boolean. True means sign detectoin will use MIN_MATCH_COUNT. 
        False means sign detection will use MIN_MATCH_PERCENTAGE.
    '''
    kp_q = queryKeypoints
    des_q = queryDescriptions
    totalMatch = len(des_q) * 1.0


    # Create SIFT, SURF, or ORB objects
    kp_f, des_f = getKeypoints(frame, feature_detection_algorithm, hessianThreshold=hessianThreshold)

    # FLANN parameters
    FLANN_INDEX_KDTREE = 0

    if feature_detection_algorithm=="orb":
        FLANN_INDEX_LSH = 6
        index_params= dict(algorithm = FLANN_INDEX_LSH, table_number = 6, key_size = 12, multi_probe_level = 1)
    else:
        index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
        search_params = dict(checks=50)

    # Matching keypoints
    flann = cv2.FlannBasedMatcher(index_params, search_params)

    if (des_q is not None) and (des_f is not None) and des_f.shape[0] > 10:
        matches = flann.knnMatch(des_q, des_f, k=2)
        # print ("des_q size: " + str(des_q.shape))
        # print ("des_f size: " + str(des_f.shape))
    else:
        matches = []

    # store good matches via Lowe's ratio test
    good = []
    for m,n in matches:
        if m.distance < 0.7*n.distance:
            good.append(m)

    print ("Match Percentage: " + str('{0:.3g}'.format(100*(len(good)/totalMatch))) + "%")

    # Checking if object is detected
    if MIN_MATCH == False:
        if len(good)/totalMatch > MIN_MATCH_PERCENTAGE:
            return True
        else:
            return False
    else:
        if len(good)>MIN_MATCH_COUNT:
            return True
        else:
            return False

#################################################################################
        
#def draw_checkerboard_center(img):




video = cv2.VideoCapture(0)

oneWaySign = cv2.imread("one_way.png")
oneWaySign = shrink(oneWaySign, 0.2)

key_pts, descrip = getKeypoints(oneWaySign, "surf")

checkerboard_check, sign_check = False, False


while cv2.waitKey(200) & 0xFF != 27:
    frame = video.read()[1]
    if frame is not None:
        frame = cv2.resize(frame, (640,480))  # Uncomment this line if issues arise
        if signDetection(frame, key_pts, descrip, "surf"):
            # do sth 
            if sign_check:
                print("Sign")
                pass 
            else:
                sign_check = True
        else:
            x, y = find_checkerboard_center(frame)
            print(x, y)
            if x != 0 and y != 0:
                # do something
                if checkerboard_check:
                    print("Checkerboard")
                    pass 
                else:
                    checkerboard_check = True
            else:
                pathSelection(frame, colorRange)
                print("Line")
            
cv2.destroyAllWindows()
cv2.waitKey(1)

Match Percentage: 1.27%
(0, 0)
(array([70, 45, 30]), array([125, 140, 215]))
Line
Match Percentage: 0.955%
(0, 0)
(array([70, 45, 30]), array([125, 140, 215]))
Line
Match Percentage: 4.14%
(0, 0)
(array([70, 45, 30]), array([125, 140, 215]))
Line
Match Percentage: 1.59%
(0, 0)
(array([70, 45, 30]), array([125, 140, 215]))
Line
Match Percentage: 3.82%
(0, 0)
(array([70, 45, 30]), array([125, 140, 215]))
Line
Match Percentage: 0.955%
(0, 0)
(array([70, 45, 30]), array([125, 140, 215]))
Line
Match Percentage: 2.23%
(0, 0)
(array([70, 45, 30]), array([125, 140, 215]))
Line
Match Percentage: 1.59%
(0, 0)
(array([70, 45, 30]), array([125, 140, 215]))
Line
Match Percentage: 1.91%
(0, 0)
(array([70, 45, 30]), array([125, 140, 215]))
Line
Match Percentage: 4.46%
(0, 0)
(array([70, 45, 30]), array([125, 140, 215]))
Line
Match Percentage: 4.14%
